# 1 - install dependencies

In [2]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time


# 2 - keypoints using mp holisric

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
# model here is holistic surrounding the code
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    

In [5]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_styled_landmarks(image,results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
path = "test_autsl/signer0_sample2_color.mp4"
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        ret,frame = cap.read()
        
        if not ret:
            break
        
        image,results = mediapipe_detection(frame,holistic)
        
        draw_styled_landmarks(image,results)
        cv2.imshow("feed",image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
        

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# 3 - extract the key  points

In [8]:
print(len(results.right_hand_landmarks.landmark))
print(len(results.pose_landmarks.landmark))

21
33


In [9]:
num_hand_marks = 21
num_pose_marks = 33


In [10]:
# how to extract the pose
# pose is on shape [x,y,z,visibility]
pose = []
for res in results.pose_landmarks.landmark:
    pose.append([res.x,res.y,res.z,res.visibility])
pose = np.array(pose,dtype=np.float16)
print(pose)

[[ 4.8584e-01  4.8145e-01 -8.2861e-01  1.0000e+00]
 [ 5.0635e-01  4.2358e-01 -8.0664e-01  1.0000e+00]
 [ 5.1904e-01  4.2041e-01 -8.0664e-01  9.9951e-01]
 [ 5.3027e-01  4.1895e-01 -8.0664e-01  9.9951e-01]
 [ 4.5679e-01  4.3335e-01 -8.0078e-01  1.0000e+00]
 [ 4.3799e-01  4.3652e-01 -8.0029e-01  1.0000e+00]
 [ 4.2188e-01  4.3896e-01 -8.0029e-01  1.0000e+00]
 [ 5.4492e-01  4.2480e-01 -5.0488e-01  9.9951e-01]
 [ 4.0234e-01  4.4922e-01 -4.5752e-01  1.0000e+00]
 [ 5.1025e-01  5.2539e-01 -7.0605e-01  1.0000e+00]
 [ 4.5068e-01  5.4346e-01 -6.9385e-01  1.0000e+00]
 [ 6.6846e-01  6.3086e-01 -2.6904e-01  9.9951e-01]
 [ 3.1128e-01  6.8359e-01 -2.6245e-01  1.0000e+00]
 [ 7.7881e-01  9.3555e-01 -1.2451e-01  8.2031e-01]
 [ 1.0333e-01  1.0049e+00 -6.7285e-01  9.5117e-01]
 [ 8.3594e-01  1.2588e+00 -1.4905e-01  6.4600e-01]
 [ 1.8530e-01  6.8066e-01 -1.3076e+00  9.3994e-01]
 [ 8.6182e-01  1.3457e+00 -1.7737e-01  6.3086e-01]
 [ 1.9214e-01  5.1465e-01 -1.4424e+00  9.2139e-01]
 [ 8.2422e-01  1.3447e+00 -2.05

In [11]:
# how to extract any hand 
# hand is on shape [x,y,z]
hand = []
for res in results.right_hand_landmarks.landmark:
    hand.append([res.x,res.y,res.z])
hand = np.array(hand,dtype=np.float16)
print(hand)

[[ 1.825e-01  6.665e-01  2.980e-07]
 [ 2.365e-01  6.372e-01 -1.562e-02]
 [ 2.791e-01  5.713e-01 -1.898e-02]
 [ 3.037e-01  5.166e-01 -2.275e-02]
 [ 3.237e-01  4.761e-01 -2.614e-02]
 [ 2.465e-01  4.673e-01  9.508e-04]
 [ 2.610e-01  3.992e-01 -1.088e-02]
 [ 2.749e-01  3.618e-01 -2.284e-02]
 [ 2.893e-01  3.337e-01 -3.134e-02]
 [ 2.200e-01  4.597e-01 -1.825e-03]
 [ 2.335e-01  3.870e-01 -1.409e-02]
 [ 2.546e-01  3.538e-01 -2.664e-02]
 [ 2.769e-01  3.335e-01 -3.528e-02]
 [ 1.923e-01  4.658e-01 -8.392e-03]
 [ 2.007e-01  3.931e-01 -2.287e-02]
 [ 2.192e-01  3.650e-01 -3.323e-02]
 [ 2.413e-01  3.491e-01 -3.958e-02]
 [ 1.608e-01  4.829e-01 -1.695e-02]
 [ 1.560e-01  4.224e-01 -3.024e-02]
 [ 1.597e-01  3.794e-01 -3.793e-02]
 [ 1.687e-01  3.430e-01 -4.309e-02]]


In [12]:
def extract_keypoints(results):
    # extract pose marks
    if results.pose_landmarks:
        pose = np.array([ [res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark ]).flatten()
    else:
        pose = np.zeros(num_pose_marks*4)
    
    # extract left hand
    if results.left_hand_landmarks:
        left_hand = np.array([ [res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        left_hand = np.zeros(num_hand_marks*3)
        
        
    # extract right hand
    if results.right_hand_landmarks:
        right_hand = np.array([ [res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark ]).flatten()
    else:
        right_hand = np.zeros(num_hand_marks*3)
    
    return np.concatenate([pose,left_hand,right_hand])
    

In [13]:
points = extract_keypoints(results)
points

array([ 4.85895634e-01,  4.81487542e-01, -8.28841984e-01,  9.99926150e-01,
        5.06155670e-01,  4.23509181e-01, -8.06652427e-01,  9.99782443e-01,
        5.18968403e-01,  4.20390815e-01, -8.06411147e-01,  9.99740660e-01,
        5.30090749e-01,  4.18863773e-01, -8.06730390e-01,  9.99721587e-01,
        4.56697851e-01,  4.33423489e-01, -8.00679088e-01,  9.99831676e-01,
        4.37955201e-01,  4.36416090e-01, -8.00284564e-01,  9.99831498e-01,
        4.21899855e-01,  4.38893616e-01, -8.00440490e-01,  9.99857008e-01,
        5.45153201e-01,  4.24816489e-01, -5.05045772e-01,  9.99753058e-01,
        4.02429909e-01,  4.49340224e-01, -4.57593530e-01,  9.99908447e-01,
        5.10257065e-01,  5.25608480e-01, -7.06103325e-01,  9.99918222e-01,
        4.50563371e-01,  5.43490648e-01, -6.93754315e-01,  9.99949157e-01,
        6.68595135e-01,  6.31097198e-01, -2.69066304e-01,  9.99702930e-01,
        3.11260492e-01,  6.83761477e-01, -2.62357712e-01,  9.99841869e-01,
        7.78686225e-01,  

# 4 - setup folders for collections

In [14]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [16]:
for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5 - collect keypoints values

In [17]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()

# 6 - data processing

In [19]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}



In [20]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [21]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(90, 30, 258)

In [29]:
X = np.array(sequences)
Y = to_categorical(labels)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

In [38]:
print(
X_train.shape,
X_test.shape,
y_train.shape,
y_test.shape,
    
)

(81, 30, 258) (9, 30, 258) (81, 3) (9, 3)


# 7 - build the model

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

In [42]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [46]:
len(actions)

3

In [51]:
input_layer = Input(shape=(30,258))
layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
layer = LSTM(128,return_sequences=True,activation="relu")(layer)
layer = LSTM(64,return_sequences=False,activation="relu")(layer)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
layer = Dense(len(actions),activation="softmax")(layer)

model = Model(inputs=input_layer,outputs=layer)


In [52]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30, 258)]         0         
                                                                 
 lstm_6 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 3)                 99  

In [53]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback],validation_data=(X_test,y_test))

Epoch 1/2000
3/3 [==============================] - 1s 210ms/step - loss: 0.5436 - categorical_accuracy: 0.6790 - val_loss: 0.5631 - val_categorical_accuracy: 0.4444
Epoch 2/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.5872 - categorical_accuracy: 0.6914 - val_loss: 0.5470 - val_categorical_accuracy: 0.4444
Epoch 3/2000
3/3 [==============================] - 0s 77ms/step - loss: 0.5755 - categorical_accuracy: 0.6914 - val_loss: 0.5508 - val_categorical_accuracy: 0.4444
Epoch 4/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.5288 - categorical_accuracy: 0.6914 - val_loss: 0.6846 - val_categorical_accuracy: 0.4444
Epoch 5/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.5791 - categorical_accuracy: 0.7284 - val_loss: 0.6783 - val_categorical_accuracy: 0.6667
Epoch 6/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.4976 - categorical_accuracy: 0.8025 - val_loss: 0.7308 - val_categorical_accuracy: 0.6667
Epoch 7/2

3/3 [==============================] - 0s 78ms/step - loss: 0.6689 - categorical_accuracy: 0.6543 - val_loss: 0.7553 - val_categorical_accuracy: 0.6667
Epoch 51/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.6494 - categorical_accuracy: 0.6543 - val_loss: 1.0356 - val_categorical_accuracy: 0.4444
Epoch 52/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.6638 - categorical_accuracy: 0.6667 - val_loss: 0.5373 - val_categorical_accuracy: 0.8889
Epoch 53/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.7156 - categorical_accuracy: 0.6790 - val_loss: 1.0985 - val_categorical_accuracy: 0.5556
Epoch 54/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.6746 - categorical_accuracy: 0.6543 - val_loss: 0.6294 - val_categorical_accuracy: 0.6667
Epoch 55/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.7230 - categorical_accuracy: 0.7037 - val_loss: 0.8282 - val_categorical_accuracy: 0.8889
Epoch 56/2000
3/3 

3/3 [==============================] - 0s 75ms/step - loss: 0.4207 - categorical_accuracy: 0.8025 - val_loss: 0.6078 - val_categorical_accuracy: 0.7778
Epoch 100/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.3380 - categorical_accuracy: 0.8395 - val_loss: 0.0888 - val_categorical_accuracy: 1.0000
Epoch 101/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.3655 - categorical_accuracy: 0.8395 - val_loss: 0.5104 - val_categorical_accuracy: 0.8889
Epoch 102/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.2279 - categorical_accuracy: 0.9383 - val_loss: 0.2025 - val_categorical_accuracy: 0.8889
Epoch 103/2000
3/3 [==============================] - 0s 81ms/step - loss: 0.2491 - categorical_accuracy: 0.9506 - val_loss: 0.4428 - val_categorical_accuracy: 0.8889
Epoch 104/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.2256 - categorical_accuracy: 0.9259 - val_loss: 0.3638 - val_categorical_accuracy: 0.8889
Epoch 105/200

3/3 [==============================] - 0s 76ms/step - loss: 0.0497 - categorical_accuracy: 0.9877 - val_loss: 0.0534 - val_categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.0417 - categorical_accuracy: 0.9877 - val_loss: 0.0493 - val_categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.0413 - categorical_accuracy: 0.9877 - val_loss: 0.0960 - val_categorical_accuracy: 1.0000
Epoch 151/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.0623 - categorical_accuracy: 0.9877 - val_loss: 0.0501 - val_categorical_accuracy: 1.0000
Epoch 152/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.0538 - categorical_accuracy: 0.9877 - val_loss: 0.0354 - val_categorical_accuracy: 1.0000
Epoch 153/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.0555 - categorical_accuracy: 0.9753 - val_loss: 0.6743 - val_categorical_accuracy: 0.8889
Epoch 154/200

3/3 [==============================] - 0s 77ms/step - loss: 0.0290 - categorical_accuracy: 1.0000 - val_loss: 0.4195 - val_categorical_accuracy: 0.8889
Epoch 198/2000
3/3 [==============================] - 0s 84ms/step - loss: 0.0223 - categorical_accuracy: 1.0000 - val_loss: 0.4368 - val_categorical_accuracy: 0.7778
Epoch 199/2000
3/3 [==============================] - 0s 86ms/step - loss: 0.0189 - categorical_accuracy: 1.0000 - val_loss: 0.4311 - val_categorical_accuracy: 0.7778
Epoch 200/2000
3/3 [==============================] - 0s 77ms/step - loss: 0.0195 - categorical_accuracy: 0.9877 - val_loss: 0.2592 - val_categorical_accuracy: 0.8889
Epoch 201/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.0158 - categorical_accuracy: 1.0000 - val_loss: 0.2410 - val_categorical_accuracy: 0.8889
Epoch 202/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.0126 - categorical_accuracy: 1.0000 - val_loss: 0.2717 - val_categorical_accuracy: 0.8889
Epoch 203/200

KeyboardInterrupt: 

In [56]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30, 258)]         0         
                                                                 
 lstm_6 (LSTM)               (None, 30, 64)            82688     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 3)                 99  

# 8 - Make predictions

In [57]:
res = model.predict(X_test)

In [60]:
[(np.argmax(res[i]),np.argmax(y_test[i])) for i in range(len(res))]

[(2, 2), (2, 2), (1, 1), (2, 2), (2, 1), (1, 1), (0, 0), (2, 2), (2, 2)]

# 9 - save weights

In [61]:
model.save('weights.h5')

In [ ]:
model.load_weights('weights.h5')

# 10 - confusion matrix evaluations

In [62]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [63]:
yhat = model.predict(X_test)

In [64]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [66]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[8, 0],
        [0, 1]],

       [[6, 0],
        [1, 2]],

       [[3, 1],
        [0, 5]]])

In [65]:
accuracy_score(ytrue, yhat)

0.8888888888888888

# 11 - test in real time

In [67]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



In [69]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.S

iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_bas

hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.sol

hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOu

thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution

thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution

<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.SolutionOutputs'>
thanks
<class 'mediapipe.python.solution_base.S

hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.Solutio